In [1]:
# SCS Fee Estimate Generator
# This program generates fee estimate letters with entity-specific logos and fee ranges

# Install required packages if not available
import sys
import subprocess

def install_package(package):
    """Install a package if it's not already installed"""
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages
required_packages = [
    "pandas>=2.1",
    "matplotlib>=3.5", 
    "Pillow>=9.0",
    "numpy>=1.20",
    "ipywidgets>=8.0",
    "reportlab>=4.0",
    "PyPDF2>=3.0",
    "openpyxl>=3.0"
]

for package in required_packages:
    install_package(package)

# Now import the packages
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
import os
from PIL import Image
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML

# SCS Fee Estimate Entity Configuration
SCS_ENTITY_CONFIG = {
    "SCOPES HEALTH INC.": {
        "name": "SCOPES Health, Inc.",
        "display_name": "SCOPES Health, Inc. (Professional)",
        "address": "9531 S. Santa Monica Blvd, Suite #1413",
        "city_state_zip": "Beverly Hills, CA 90210-4503",
        "tax_id": "Tax I.D. #99-1006225",
        "phone": "Billing Office: (310) 469-5111",
        "email": "billing@scopeshealthcare.com",
        "logo_path": "../Material/scopes_logo.jpg",
        "billing_team": "SCOPES Health Billing Team",
        "service_type": "Professional Medical Services",
        "fee_ranges": {
            "professional": "$19k-$22k",
            "anesthesia": "$2k-$4k", 
            "facility": "$74k-$78k"
        }
    },
    "UNIVERSITY SURGICAL INSTITUTE, LLC": {
        "name": "University Surgical Institute, LLC",
        "display_name": "University Surgical Institute, LLC (ASC)",
        "address": "8200 Stockdale HWY, Suite M-10287",
        "city_state_zip": "Bakersfield, CA 93311",
        "tax_id": "Tax I.D. #93-2514800",
        "phone": "Billing Office: (310) 469-5111",
        "email": "billing@universitysurgicalinstitute.com",
        "logo_path": "../Material/usi_logo.png",
        "billing_team": "University Surgical Institute Billing Team",
        "service_type": "Surgical Facility Services",
        "fee_ranges": {
            "professional": "$19k-$22k",
            "anesthesia": "$2k-$4k", 
            "facility": "$74k-$78k"
        }
    },
    "ANESTHESIOLOGY & PERIOPERATIVE MEDICINE SPECIALISTS": {
        "name": "Anesthesiology & Perioperative Medicine Specialists",
        "display_name": "APMS, Inc. (Anesthesia)",
        "address": "9531 S. Santa Monica Blvd, Suite #1413",
        "city_state_zip": "Beverly Hills, CA 90210-4503",
        "tax_id": "Tax I.D. #99-1043743",
        "phone": "Billing Office: (310) 469-5111",
        "email": "apmsmedicalbilling@gmail.com",
        "logo_path": "../Material/apms_logo.jpg",
        "billing_team": "APMS Billing Team",
        "service_type": "Anesthesia Services",
        "fee_ranges": {
            "professional": "$19k-$22k",
            "anesthesia": "$2k-$4k", 
            "facility": "$74k-$78k"
        }
    }
}


Installing pandas>=2.1...
Installing matplotlib>=3.5...
Installing Pillow>=9.0...
Installing numpy>=1.20...
Installing ipywidgets>=8.0...
Installing reportlab>=4.0...
Installing PyPDF2>=3.0...
Installing openpyxl>=3.0...


In [2]:
# Load main datasets
CombinedCases = pd.read_excel("../DataBase/CombinedCasesFinal.xlsx")
PI_Attorney = pd.read_csv("../DataBase/PI Attorney.csv")
PI_Patient = pd.read_csv("../DataBase/PI Patients.csv")
Case_Rep = pd.read_csv("../DataBase/Case Rep.csv")

print("Data loaded successfully!")
print(f"CombinedCases: {CombinedCases.shape}")
print(f"PI_Attorney: {PI_Attorney.shape}")
print(f"PI_Patient: {PI_Patient.shape}")
print(f"Case_Rep: {Case_Rep.shape}")

# Check the actual column names
print(f"\nCombinedCases columns: {list(CombinedCases.columns)}")
print(f"PI_Attorney columns: {list(PI_Attorney.columns)}")
print(f"PI_Patient columns: {list(PI_Patient.columns)}")
print(f"Case_Rep columns: {list(Case_Rep.columns)}")


Data loaded successfully!
CombinedCases: (497, 25)
PI_Attorney: (61, 11)
PI_Patient: (102, 15)
Case_Rep: (75, 4)

CombinedCases columns: ['First Name', 'Last Name', 'Service Type', 'Physcian Name', 'Date of Service', 'Location Name', 'Date Billed', 'Service Provided', 'Full Service Description', 'Law Firm', 'Attorney', 'Case Rep', 'Item Charge', 'Pmt Date', 'Paid Amount', 'Collection %', 'Lien Reduction %', 'Settlement Accepted', 'Discount Date', 'Discount Amount', 'Case Status', 'Charge Amount', 'Settlement %', 'Settlement Date', 'Entity']
PI_Attorney columns: ['Attorney Name', 'Firm', 'Address', 'City', 'State', 'Zip', 'Work Phone', 'Mobile Phone', 'Assistant Name', 'Email', 'Asst Phone']
PI_Patient columns: ['Id', 'Patient Fname', 'Patient Lname', 'DOB', 'Patient Phone', 'Address', 'City', 'State', 'Zip', 'Physcians', 'Case Reps', 'Attorneys', 'DOI', 'Patient Email', 'Patient ID']
Case_Rep columns: ['Case Rep Name', 'CR Email', 'CR Phone', 'CR Fax']


In [3]:
# Data processing and merging functions (adapted from BillsProgram_Clean.ipynb)
TRACE_ENABLED = False

def set_trace(enabled: bool):
    global TRACE_ENABLED
    TRACE_ENABLED = bool(enabled)

_trace_events = []
_trace_store = {}

def trace(event: str, **kwargs):
    if TRACE_ENABLED:
        _trace_events.append({"event": event, **kwargs})

def normalize_text(s):
    """Lowercase, strip, collapse whitespace, safe if series contains NaN."""
    return s.fillna('').astype(str).str.strip().str.lower().str.replace(r'\s+', ' ', regex=True)

def clean_money(series):
    """Remove $ , and convert to float. Returns float series (NaN if not convertible)."""
    return pd.to_numeric(series.astype(str).str.replace(r'[\$,]', '', regex=True).str.replace('--','').str.strip(), errors='coerce')

def parse_dates(df, cols):
    """Inplace parse of multiple date columns; returns df."""
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors='coerce', infer_datetime_format=True)
    return df

def _safe_to_datetime(val):
    """Parse dates robustly, handling MM/DD/YYYY, DD/MM/YYYY, and dash variants."""
    try:
        # Fast path for pandas Timestamp or datetime
        if hasattr(val, 'to_pydatetime') or hasattr(val, 'year'):
            return pd.to_datetime(val, errors='coerce')
        s = str(val).strip()
        if not s or s.lower() in {"nan", "none", "nat"}:
            return pd.NaT
        # Normalize separators
        sep = "/" if "/" in s else ("-" if "-" in s else None)
        if sep:
            parts = s.split(sep)
            if len(parts) == 3:
                try:
                    p0 = int(parts[0])
                    p1 = int(parts[1])
                except Exception:
                    # Fallback to general parser
                    return pd.to_datetime(s, errors='coerce', dayfirst=False)
                # Decide dayfirst
                if p0 > 12 and p1 <= 12:
                    return pd.to_datetime(s, errors='coerce', dayfirst=True)
                if p1 > 12 and p0 <= 12:
                    return pd.to_datetime(s, errors='coerce', dayfirst=False)
                # Ambiguous (both <= 12): try month-first then day-first if needed
                dt = pd.to_datetime(s, errors='coerce', dayfirst=False)
                if pd.isna(dt):
                    dt = pd.to_datetime(s, errors='coerce', dayfirst=True)
                return dt
        # Fallback generic
        return pd.to_datetime(s, errors='coerce', dayfirst=False)
    except Exception:
        return pd.NaT

def combined_parse_dates(df, cols):
    """Inplace parse of multiple date columns; returns df."""
    for c in cols:
        if c in df.columns:
            df[c] = df[c].apply(_safe_to_datetime)
    return df

def prepare_combined(df):
    df = df.copy()
    df['patient_full'] = normalize_text(df['First Name']) + ' ' + normalize_text(df['Last Name'])
    if 'Attorney' in df.columns:
        df['attorney_norm'] = normalize_text(df['Attorney'])
    combined_parse_dates(df, ['Date of Service', 'Date Billed', 'Pmt Date', 'Discount Date', 'Settlement Date'])
    for col in ['Item Charge', 'Paid Amount', 'Charge Amount', 'Settlement Accepted', 'Discount Amount', 'Collection %', 'Lien Reduction %', 'Settlement %']:
        if col in df.columns:
            df[col] = clean_money(df[col])
    if 'Date Billed' in df.columns:
        df['billing_period'] = df['Date Billed'].dt.to_period('M').astype(str).fillna('unbilled')
    else:
        df['billing_period'] = 'unbilled'
    return df

def prepare_patients(df):
    df = df.copy()
    df['patient_full'] = normalize_text(df['Patient Fname']) + ' ' + normalize_text(df['Patient Lname'])
    parse_dates(df, ['DOB', 'DOI'])
    if 'Patient Phone' in df.columns:
        df['Patient Phone'] = df['Patient Phone'].astype(str).replace('nan','',regex=False)
        # Format phone numbers to (XXX) XXX-XXXX format
        def format_phone(phone_str):
            if pd.isna(phone_str) or phone_str == '' or phone_str == 'nan':
                return 'Unknown'
            # Remove all non-digit characters
            digits = ''.join(filter(str.isdigit, str(phone_str)))
            if len(digits) == 10:
                return f"({digits[:3]}) {digits[3:6]}-{digits[6:]}"
            elif len(digits) == 11 and digits[0] == '1':
                return f"({digits[1:4]}) {digits[4:7]}-{digits[7:]}"
            else:
                return str(phone_str)  # Return original if can't format
        df['Patient Phone'] = df['Patient Phone'].apply(format_phone)
    # Ensure ZIP is a clean string (no .0)
    if 'Zip' in df.columns:
        def clean_zip(z):
            if pd.isna(z):
                return ''
            s = str(z).strip()
            # If numeric like 93307.0 -> 93307
            try:
                f = float(s)
                if f.is_integer():
                    return str(int(f))
            except Exception:
                pass
            # Remove trailing .0 if present
            if s.endswith('.0'):
                s = s[:-2]
            return s
        df['Zip'] = df['Zip'].apply(clean_zip)
    return df

def prepare_attorneys(df):
    df = df.copy()
    df['attorney_norm'] = normalize_text(df['Attorney Name'])
    if 'Zip' in df.columns:
        df['Zip'] = df['Zip'].apply(lambda z: str(int(z)) if (pd.notna(z) and float(z).is_integer()) else (str(z) if pd.notna(z) else ''))
    return df

def prepare_case_reps(df):
    df = df.copy()
    df['case_rep_norm'] = normalize_text(df['Case Rep Name'])
    return df

def merge_all(combined, patients, attorneys, case_reps):
    """Returns merged dataframe with patient, attorney, and case rep fields attached."""
    c = prepare_combined(combined)
    p = prepare_patients(patients)
    a = prepare_attorneys(attorneys)
    cr = prepare_case_reps(case_reps)

    patient_cols = {}
    for col in ['DOB', 'DOI', 'Patient Phone', 'Address', 'City', 'State', 'Zip', 'Patient Email']:
        if col in p.columns:
            if col == 'Address':
                patient_cols[col] = 'patient_street'
            else:
                patient_cols[col] = f'patient_{col.lower().replace(" ", "_")}'
    
    # Create a combined address field
    if 'Address' in p.columns and 'City' in p.columns and 'State' in p.columns and 'Zip' in p.columns:
        # Create address for each row individually
        def create_address(row):
            address = str(row['Address']) if pd.notna(row['Address']) else ''
            city = str(row['City']) if pd.notna(row['City']) else ''
            state = str(row['State']) if pd.notna(row['State']) else ''
            zip_code = str(row['Zip']) if pd.notna(row['Zip']) else ''
            
            # Clean up the values
            address = address.replace('nan', '').replace('None', '').strip()
            city = city.replace('nan', '').replace('None', '').strip()
            state = state.replace('nan', '').replace('None', '').strip()
            zip_code = zip_code.replace('nan', '').replace('None', '').strip()
            
            # Combine parts
            parts = [part for part in [address, city, state, zip_code] if part]
            return ', '.join(parts) if parts else 'Unknown Address'
        
        p['patient_address'] = p.apply(create_address, axis=1).astype(str)
        patient_cols['patient_address'] = 'patient_address'
    p_rename = p[['patient_full'] + list(patient_cols.keys())].rename(columns=patient_cols)

    merged = c.merge(p_rename, on='patient_full', how='left')

    att_pick = []
    att_map = {}
    for col in ['Attorney Name', 'Firm', 'Address', 'City', 'State', 'Zip', 'Work Phone', 'Email']:
        if col in a.columns:
            att_pick.append(col)
            att_map[col] = 'attorney_' + col.lower().replace(' ', '_')

    a_small = a[['attorney_norm'] + att_pick].rename(columns=att_map)
    merged = merged.merge(a_small, on='attorney_norm', how='left')
    
    # Add Case Rep information
    if 'Case Rep' in merged.columns:
        merged['case_rep_norm'] = normalize_text(merged['Case Rep'])
        cr_pick = []
        cr_map = {}
        for col in ['Case Rep Name', 'CR Email', 'CR Phone', 'CR Fax']:
            if col in cr.columns:
                cr_pick.append(col)
                cr_map[col] = 'case_rep_' + col.lower().replace(' ', '_').replace('cr_', '')
        
        cr_small = cr[['case_rep_norm'] + cr_pick].rename(columns=cr_map)
        merged = merged.merge(cr_small, on='case_rep_norm', how='left')
    
    trace('merge_all_complete', merged_shape=tuple(merged.shape))
    return merged

# Merge all data
merged = merge_all(CombinedCases, PI_Patient, PI_Attorney, Case_Rep)
print(f"Merged data: {merged.shape}")
print(f"Data loaded successfully!")
print(f"Found {len(merged)} records")
print(f"Entities: {list(merged['Entity'].dropna().unique())}")


Merged data: (520, 50)
Data loaded successfully!
Found 520 records
Entities: ['SCOPES HEALTH INC.', 'UNIVERSITY SURGICAL INSTITUTE, LLC', 'ANESTHESIOLOGY & PERIOPERATIVE MEDICINE SPECIALISTS']


C:\Users\Mazen\AppData\Local\Temp\ipykernel_5832\2499219226.py:27: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[c] = pd.to_datetime(df[c], errors='coerce', infer_datetime_format=True)
C:\Users\Mazen\AppData\Local\Temp\ipykernel_5832\2499219226.py:27: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[c] = pd.to_datetime(df[c], errors='coerce', infer_datetime_format=True)


In [4]:
def get_patient_data_for_fee_estimate(merged_df, first_name, last_name, dob, entity=None):
    """Get patient data for fee estimate generation."""
    import pandas as pd
    
    first_name_norm = first_name.strip().lower()
    last_name_norm = last_name.strip().lower()
    
    if isinstance(dob, str):
        try:
            dob_parsed = pd.to_datetime(dob, format='%m/%d/%Y')
        except Exception:
            try:
                dob_parsed = pd.to_datetime(dob)
            except Exception:
                return None, None, None
    else:
        dob_parsed = dob
    
    mask = (
        (merged_df['First Name'].str.lower().str.strip() == first_name_norm) &
        (merged_df['Last Name'].str.lower().str.strip() == last_name_norm) &
        (pd.to_datetime(merged_df['patient_dob'], errors='coerce') == dob_parsed)
    )
    
    if entity and str(entity).strip():
        mask &= (merged_df['Entity'].astype(str).str.strip().str.lower() == str(entity).strip().lower())
    
    patient_data = merged_df[mask]
    
    if patient_data.empty:
        print(f"No patient found with name: {first_name} {last_name}, DOB: {dob}, Entity: {entity or 'Any'}")
        return None, None, None
    
    # Use the most recent row by Date of Service
    try:
        patient_data_sorted = patient_data.sort_values(
            by=['Date of Service'],
            key=lambda s: s.apply(_safe_to_datetime)
        ).reset_index(drop=True)
        first_row = patient_data_sorted.iloc[-1]
    except Exception:
        first_row = patient_data.iloc[-1]
    
    # Get patient address as string
    patient_address = first_row.get('patient_address', 'Unknown Address')
    if hasattr(patient_address, 'iloc'):
        patient_address = str(patient_address.iloc[0]) if len(patient_address) > 0 else 'Unknown Address'
    else:
        patient_address = str(patient_address)
    
    patient_info = {
        'Patient Fname': first_row.get('First Name', 'Unknown'),
        'Patient Lname': first_row.get('Last Name', 'Unknown'),
        'DOB': first_row.get('patient_dob', 'Unknown'),
        'DOI': first_row.get('patient_doi', 'Unknown'),
        'Patient Phone': first_row.get('patient_patient_phone', 'Unknown'),
        'patient_address': patient_address,
        'patient_street': first_row.get('patient_street', ''),
        'patient_city': first_row.get('patient_city', ''),
        'patient_state': first_row.get('patient_state', ''),
        'patient_zip': first_row.get('patient_zip', ''),
        'Entity': first_row.get('Entity', 'Unknown Entity')
    }
    
    attorney_info = {
        'Attorney Name': first_row.get('attorney_attorney_name', first_row.get('Attorney', 'Unknown Attorney')),
        'Case Rep': first_row.get('Case Rep', 'Unknown'),
        'Firm': first_row.get('attorney_firm', first_row.get('Law Firm', 'Unknown Firm')),
        'Address': first_row.get('attorney_address', 'Unknown Address'),
        'City': first_row.get('attorney_city', 'Unknown City'),
        'State': first_row.get('attorney_state', 'Unknown State'),
        'Zip': first_row.get('attorney_zip', 'Unknown Zip'),
        'attorney_email': first_row.get('attorney_email', ''),
        'case_rep_email': first_row.get('case_rep_email', ''),
        'case_rep_phone': first_row.get('case_rep_phone', ''),
        'case_rep_fax': first_row.get('case_rep_fax', '')
    }
    
    return patient_info, attorney_info, first_row.get('Entity', 'Unknown Entity')


In [5]:
def generate_fee_estimate_pdf(patient_info, attorney_info, entity_name, output_path="fee_estimate.pdf"):
    """Generate a PDF fee estimate letter matching the attached design."""
    try:
        from reportlab.lib.pagesizes import letter
        from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
        from reportlab.lib import colors
        from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
        from reportlab.lib.units import inch
        from PyPDF2 import PdfReader, PdfWriter
        from reportlab.pdfgen import canvas
        from reportlab.lib.utils import ImageReader
        import io
    except ImportError:
        print("Installing required packages...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "reportlab", "PyPDF2"])
        from reportlab.lib.pagesizes import letter
        from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
        from reportlab.lib import colors
        from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
        from reportlab.lib.units import inch
        from PyPDF2 import PdfReader, PdfWriter
        from reportlab.pdfgen import canvas
        from reportlab.lib.utils import ImageReader
        import io
    
    # Create the fee estimate content
    doc = SimpleDocTemplate(output_path, pagesize=letter, 
                            rightMargin=0.75*inch, leftMargin=0.75*inch,
                            topMargin=0.75*inch, bottomMargin=0.75*inch)
    styles = getSampleStyleSheet()
    elements = []
    
    # Create custom styles for proper formatting
    header_style = ParagraphStyle(
        'HeaderStyle',
        parent=styles['Normal'],
        fontSize=10,
        fontName='Helvetica-Bold',
        leading=12,
        leftIndent=0,
        rightIndent=0,
        alignment=0,  # Left align
        spaceBefore=0,
        spaceAfter=0
    )
    
    normal_style = ParagraphStyle(
        'NormalStyle',
        parent=styles['Normal'],
        fontSize=10,
        fontName='Helvetica',
        leading=12,
        leftIndent=0,
        rightIndent=0,
        alignment=0,  # Left align
        spaceBefore=0,
        spaceAfter=0
    )
    
    bold_style = ParagraphStyle(
        'BoldStyle',
        parent=styles['Normal'],
        fontSize=10,
        fontName='Helvetica-Bold',
        leading=12,
        leftIndent=0,
        rightIndent=0,
        alignment=0,  # Left align
        spaceBefore=0,
        spaceAfter=0
    )
    
    # Get entity configuration
    if entity_name in SCS_ENTITY_CONFIG:
        config = SCS_ENTITY_CONFIG[entity_name]
    else:
        # Default to SCOPES if entity not found
        config = SCS_ENTITY_CONFIG["SCOPES HEALTH INC."]
    
    # Header Section - Left side with company info, Right side with logos
    header_text = f"""
    <b>{config['display_name']}</b><br/>
    {config['address']}<br/>
    {config['city_state_zip']}<br/><br/>
    {config['tax_id']}<br/>
    {config['phone']}<br/>
    <u>{config['email']}</u>
    """
    
    # Add logos on the right side
    logo_path = config.get('logo_path', '')
    if logo_path and os.path.exists(logo_path):
        try:
            from PIL import Image as PILImage
            pil_img = PILImage.open(logo_path)
            img_width, img_height = pil_img.size
            
            # Scale logos to reasonable size
            max_width = 1.8 * inch
            max_height = 0.8 * inch
            
            scale_w = max_width / img_width
            scale_h = max_height / img_height
            scale = min(scale_w, scale_h)
            
            logo_width = img_width * scale
            logo_height = img_height * scale
            
            logo = Image(logo_path, width=logo_width, height=logo_height)
            logo.hAlign = 'RIGHT'
            
            header_table_data = [
                [Paragraph(header_text, header_style), logo]
            ]
            header_table = Table(
                header_table_data,
                colWidths=[4*inch, 2*inch],
                rowHeights=[1*inch]
            )
            header_table.setStyle(TableStyle([
                ("ALIGN", (0,0), (0,0), "LEFT"),
                ("ALIGN", (1,0), (1,0), "RIGHT"),
                ("VALIGN", (0,0), (-1,-1), "TOP"),
                ("LEFTPADDING", (0,0), (-1,-1), 0),
                ("RIGHTPADDING", (0,0), (-1,-1), 0),
                ("TOPPADDING", (0,0), (-1,-1), 0),
                ("BOTTOMPADDING", (0,0), (-1,-1), 0),
            ]))
            elements.append(header_table)
        except Exception as e:
            print(f"Could not load logo: {e}")
            elements.append(Paragraph(header_text, header_style))
    else:
        elements.append(Paragraph(header_text, header_style))
    
    elements.append(Spacer(1, 20))
    
    # Delivery and Recipient Information
    current_date = datetime.now().strftime("%m-%d-%Y")
    delivery_email = attorney_info.get('case_rep_email', '') or attorney_info.get('attorney_email', '') or config.get('email', '')
    
    # Create delivery information using proper paragraph styles instead of HTML
    delivery_info = [
        f"Delivery via Email only: {delivery_email}",
        f"{attorney_info.get('Attorney Name', 'Unknown Attorney')}, Esq.",
        f"Attn: {attorney_info.get('Case Rep', 'Unknown')}",
        f"{attorney_info.get('Firm', 'Unknown Firm')}",
        f"{attorney_info.get('Address', 'Unknown Address')}",
        f"{attorney_info.get('City', 'Unknown')}, {attorney_info.get('State', 'Unknown')} {attorney_info.get('Zip', 'Unknown')}"
    ]
    
    # Add each line as a separate paragraph with appropriate styling
    for i, info in enumerate(delivery_info):
        if i == 0:  # Email line - underline
            elements.append(Paragraph(f"Delivery via Email only: <u>{delivery_email}</u>", normal_style))
        elif i == 1:  # Attorney name - bold
            elements.append(Paragraph(f"<b>{attorney_info.get('Attorney Name', 'Unknown Attorney')}, Esq.</b>", bold_style))
        elif i == 3:  # Firm name - bold
            elements.append(Paragraph(f"<b>{attorney_info.get('Firm', 'Unknown Firm')}</b>", bold_style))
        else:
            elements.append(Paragraph(info, normal_style))
    elements.append(Spacer(1, 20))
    
    # Main Title
    title_style = ParagraphStyle(
        'TitleStyle',
        parent=styles['Heading1'],
        fontSize=16,
        fontName='Helvetica-Bold',
        alignment=1,  # Center
        leading=18
    )
    elements.append(Paragraph("FEE ESTIMATE", title_style))
    elements.append(Spacer(1, 20))
    
    # Introductory paragraph
    intro_text = f"""Dear {attorney_info.get('Case Rep', 'Sir/Madam')}: Pursuant to your requested fee estimate for the following case and/or individual referenced below and the recommended surgical services, the procedure fee estimates are as follows:"""
    elements.append(Paragraph(intro_text, normal_style))
    elements.append(Spacer(1, 20))
    
    # Fee Estimate Table
    patient_dob_str = patient_info.get('DOB', 'Unknown')
    if hasattr(patient_dob_str, 'strftime'):
        patient_dob_str = patient_dob_str.strftime('%m/%d/%Y')
    else:
        patient_dob_str = str(patient_dob_str)
    
    patient_phone = patient_info.get('Patient Phone', 'Unknown')
    
    # Create fee estimate table
    fee_table_data = [
        [Paragraph("Injured Party", bold_style), Paragraph("Fee Estimate", bold_style)]
    ]
    
    # Left column - Patient info
    patient_info_text = f"""
    <b>{patient_info.get('Patient Fname', 'Unknown')} {patient_info.get('Patient Lname', 'Unknown')}</b><br/>
    DOB: {patient_dob_str}<br/>
    Tel: {patient_phone}
    """
    
    # Right column - Fee estimates based on entity
    if entity_name == "SCOPES HEALTH INC.":
        fee_estimate_text = "Professional Services: $19k-$22k"
    elif entity_name == "ANESTHESIOLOGY & PERIOPERATIVE MEDICINE SPECIALISTS":
        fee_estimate_text = "Anesthesia Services: $2k-$4k"
    elif entity_name == "UNIVERSITY SURGICAL INSTITUTE, LLC":
        fee_estimate_text = "Facility Services: $74k-$78k"
    else:
        # Default case - show all ranges
        fee_estimate_text = f"""
        Professional Services: $19k-$22k,<br/>
        Anesthesia Services: $2k-$4k,<br/>
        Facility Services: $74k-$78k
        """
    
    fee_table_data.append([
        Paragraph(patient_info_text, normal_style),
        Paragraph(fee_estimate_text, normal_style)
    ])
    
    fee_table = Table(fee_table_data, colWidths=[3*inch, 3*inch])
    fee_table.setStyle(TableStyle([
        ("GRID", (0,0), (-1,-1), 0.5, colors.black),
        ("ALIGN", (0,0), (-1,-1), "LEFT"),
        ("VALIGN", (0,0), (-1,-1), "TOP"),
        ("FONTSIZE", (0,0), (-1,-1), 10),
        ("FONTNAME", (0,0), (-1,-1), "Helvetica"),
        ("LEFTPADDING", (0,0), (-1,-1), 8),
        ("RIGHTPADDING", (0,0), (-1,-1), 8),
        ("TOPPADDING", (0,0), (-1,-1), 8),
        ("BOTTOMPADDING", (0,0), (-1,-1), 8),
    ]))
    
    elements.append(fee_table)
    elements.append(Spacer(1, 20))
    
    # Concluding paragraph
    conclusion_text = """Subsequent surgical services rendered are expected to have similar fees and, if, and when
performed in the future. Please let us know if you have any questions. Also, please acknowledge
receipt of this estimate and accept this estimate by signing below and returning a signed copy of
this letter back to our billing office so we can proceed with scheduling and performing the
aforementioned services."""
    elements.append(Paragraph(conclusion_text, normal_style))
    elements.append(Spacer(1, 30))
    
    # Signature section - Left side with "Sincerely" section, Right side with "Accepted by" section
    signature_table_data = [
        ["Sincerely,", ""],
        ["Pacific Surgical Billing, LLC", ""],
        ["for SCOPES Health, USI,", ""],
        ["and APMS", ""],
        ["", ""],  # Blank line for spacing
        ["Mark R. Brown for APMS", ""],
        ["Direct: (310) 428-2254", ""],
        ["Email: MarkB@adaptpac.com", ""],
        ["", "Accepted by:"],
        ["", ""],  # Signature line will be here
        ["", f"{attorney_info.get('Case Rep', 'Unknown')}, {attorney_info.get('Firm', 'Unknown Firm')}"]
    ]
    
    signature_table = Table(signature_table_data, colWidths=[3*inch, 3*inch])
    signature_table.setStyle(TableStyle([
        ("ALIGN", (0,0), (0,-1), "LEFT"),
        ("ALIGN", (1,0), (1,-1), "LEFT"),  # Left align the right column
        ("FONTSIZE", (0,0), (-1,-1), 10),
        ("FONTNAME", (0,0), (-1,-1), "Helvetica"),
        ("FONTNAME", (1,10), (1,10), "Helvetica-Bold"),  # Bold for case rep line
        ("FONTNAME", (1,8), (1,8), "Helvetica-Bold"),  # Bold for "Accepted by:"
        ("LINEBELOW", (1,9), (1,9), 0.5, colors.black),  # Line for signature
        ("LEFTPADDING", (0,0), (-1,-1), 0),
        ("RIGHTPADDING", (0,0), (-1,-1), 0),
        ("TOPPADDING", (0,0), (-1,-1), 2),
        ("BOTTOMPADDING", (0,0), (-1,-1), 2),
    ]))
    
    elements.append(signature_table)
    
    # Build the PDF
    doc.build(elements)
    
    print(f"Fee estimate PDF generated successfully: {output_path}")
    return output_path


In [6]:
def create_fee_estimate_complete(merged_df, first_name, last_name, dob, output_path=None, entity=None):
    """Complete function to get patient data and generate a fee estimate PDF."""
    
    patient_info, attorney_info, entity_name = get_patient_data_for_fee_estimate(
        merged_df, first_name, last_name, dob, entity=entity
    )
    
    if patient_info is None:
        return None
    
    if output_path is None:
        safe_name = f"{first_name}_{last_name}_fee_estimate.pdf".replace(" ", "_")
        output_path = safe_name
    
    pdf_path = generate_fee_estimate_pdf(patient_info, attorney_info, entity_name, output_path)
    
    return {
        'patient_info': patient_info,
        'attorney_info': attorney_info, 
        'entity_name': entity_name,
        'pdf_path': pdf_path
    }


In [7]:
# Enable Jupyter widgets extension
try:
    import subprocess
    import sys
    
    # Try to enable the widget extension
    try:
        subprocess.check_call([sys.executable, "-m", "jupyter", "nbextension", "enable", "--py", "widgetsnbextension", "--sys-prefix"], 
                            capture_output=True)
        print("✅ Widget extension enabled successfully")
    except subprocess.CalledProcessError as e:
        print("⚠️  Could not enable widget extension automatically. Widgets may still work.")
        print("If widgets don't work, try restarting your Jupyter kernel.")
    except FileNotFoundError:
        print("⚠️  Jupyter command not found. Widgets may still work.")
        
except Exception as e:
    print(f"⚠️  Error enabling widgets: {e}")
    print("Widgets may still work without explicit enabling.")


⚠️  Error enabling widgets: Popen.__init__() got an unexpected keyword argument 'capture_output'
Widgets may still work without explicit enabling.


In [ ]:
# GUI Interface for Fee Estimate Generator
# Input widgets with proper copy/paste support
first_name_w = widgets.Text(
    description='First Name', 
    placeholder='e.g., Jose', 
    layout=widgets.Layout(width='200px'),
    style={'description_width': 'initial'}
)
last_name_w = widgets.Text(
    description='Last Name', 
    placeholder='e.g., Salinas', 
    layout=widgets.Layout(width='200px'),
    style={'description_width': 'initial'}
)
dob_w = widgets.Text(
    description='DOB', 
    placeholder='MM/DD/YYYY', 
    layout=widgets.Layout(width='200px'),
    style={'description_width': 'initial'}
)

# Entity dropdown
if 'Entity' in merged.columns:
    entity_values = sorted(list({str(v) for v in merged['Entity'].dropna().unique()}))
    entity_options = ['Any'] + entity_values
else:
    entity_options = ['Any']
entity_w = widgets.Dropdown(options=entity_options, value='Any', description='Entity')

generate_btn = widgets.Button(description='Generate SCS Fee Estimate', button_style='primary', icon='file')
status_out = widgets.Output()
pdf_out = widgets.Output()

def on_generate_clicked(_):
    status_out.clear_output()
    pdf_out.clear_output()
    
    with status_out:
        fn = first_name_w.value.strip()
        ln = last_name_w.value.strip()
        dob_str = dob_w.value.strip()
        entity_sel = None if entity_w.value == 'Any' else entity_w.value

        if not fn or not ln or not dob_str:
            print('Please fill First Name, Last Name, and DOB.')
            return
        
        try:
            dob = pd.to_datetime(dob_str, format='%m/%d/%Y')
        except Exception:
            try:
                dob = pd.to_datetime(dob_str)
            except Exception:
                print('DOB format invalid. Use MM/DD/YYYY.')
                return
        
        # Create output directory structure: ../Output_Folder/{PatientFirstName}_{PatientLastName}_FeeEstimates
        output_base_dir = "../Output_Folder/FeeEstimates"
        invoice_name = f"{fn}_{ln}_FeeEstimates"
        output_dir = os.path.join(output_base_dir, invoice_name)
        os.makedirs(output_dir, exist_ok=True)
        
        # Get entity name for filename
        entity_name = entity_sel or 'Any'
        entity_clean = entity_name.replace(' ', '_').replace(',', '').replace('.', '').replace('&', 'and').replace('LLC', '').replace('INC', '').strip('_')
        
        pdf_name = f"{fn}_{ln}_FeeEstimate_{entity_clean}_{datetime.now().strftime('%Y%m%d')}.pdf".replace(' ', '_')
        pdf_path = os.path.join(output_dir, pdf_name)
        
        # Generate the fee estimate
        result = create_fee_estimate_complete(merged, fn, ln, dob, output_path=pdf_path, entity=entity_sel)
        
        if not result:
            print('No matching patient found. Check name, DOB, and Entity.')
            return
        
        print(f"Fee Estimate created: {result['pdf_path']}")
        print(f"Output folder: {output_dir}")
        print(f"Entity: {entity_sel or 'Any'}")
        print(f"SCS Fee Ranges Applied:")
        print(f"  - Professional Services: $19k-$22k")
        print(f"  - Anesthesia Services: $2k-$4k")
        print(f"  - Facility Services: $74k-$78k")
        
        # Store result in a way that's accessible to the pdf_out section
        global last_result
        last_result = result
    
    with pdf_out:
        if 'last_result' in globals() and last_result:
            rel = last_result['pdf_path']
            display(HTML(f"<embed src='{rel}' type='application/pdf' width='100%' height='800px' />"))

generate_btn.on_click(on_generate_clicked)

form = widgets.VBox([
    widgets.HBox([first_name_w, last_name_w, dob_w]),
    widgets.HBox([entity_w]),
    generate_btn,
    status_out,
    pdf_out
])

display(form)


# SCS Fee Estimate Generator

This notebook generates professional fee estimate letters for SCS (Spinal Cord Stimulator) procedures with entity-specific branding and fee ranges.

## Features:
- **Entity-Specific Branding**: Each entity (SCOPES Health, University Surgical Institute, APMS) has its own logo and contact information
- **SCS Fee Ranges**: 
  - Professional Services: $19k-$22k
  - Anesthesia Services: $2k-$4k  
  - Facility Services: $74k-$78k
- **Professional Layout**: Matches the design from the reference image
- **Auto-Generated Output**: Files saved to `../Output_Folder/{PatientName}_FeeEstimates/`

## Usage:
1. Enter patient First Name, Last Name, and DOB
2. Select the entity (or "Any" for automatic detection)
3. Click "Generate SCS Fee Estimate"
4. The PDF will be generated and displayed below

## Alternative: Simple Input Version

If widgets don't work, you can use this simple function:

```python
# Simple version without widgets
def generate_fee_estimate_simple():
    print("=== SCS Fee Estimate Generator - Simple Version ===")
    
    first_name = input("Enter First Name: ").strip()
    last_name = input("Enter Last Name: ").strip()
    dob_str = input("Enter DOB (MM/DD/YYYY): ").strip()
    
    # Entity selection
    print("\nAvailable Entities:")
    entities = list(merged['Entity'].dropna().unique())
    for i, entity in enumerate(entities, 1):
        print(f"{i}. {entity}")
    print("0. Any Entity")
    
    try:
        choice = int(input("Select Entity (number): "))
        if choice == 0:
            entity_sel = None
        else:
            entity_sel = entities[choice - 1]
    except (ValueError, IndexError):
        print("Invalid choice, using 'Any Entity'")
        entity_sel = None
    
    if not first_name or not last_name or not dob_str:
        print('Please fill First Name, Last Name, and DOB.')
        return
    
    try:
        dob = pd.to_datetime(dob_str, format='%m/%d/%Y')
    except Exception:
        try:
            dob = pd.to_datetime(dob_str)
        except Exception:
            print('DOB format invalid. Use MM/DD/YYYY.')
            return
    
    # Create output directory
    output_base_dir = "../Output_Folder"
    invoice_name = f"{first_name}_{last_name}_FeeEstimates"
    output_dir = os.path.join(output_base_dir, invoice_name)
    os.makedirs(output_dir, exist_ok=True)
    
    # Generate filename
    entity_name = entity_sel or 'Any'
    entity_clean = entity_name.replace(' ', '_').replace(',', '').replace('.', '').replace('&', 'and').replace('LLC', '').replace('INC', '').strip('_')
    pdf_name = f"{first_name}_{last_name}_FeeEstimate_{entity_clean}_{datetime.now().strftime('%Y%m%d')}.pdf".replace(' ', '_')
    pdf_path = os.path.join(output_dir, pdf_name)
    
    # Generate the fee estimate
    result = create_fee_estimate_complete(merged, first_name, last_name, dob, output_path=pdf_path, entity=entity_sel)
    
    if not result:
        print('No matching patient found. Check name, DOB, and Entity.')
        return
    
    print(f"\n✅ Fee Estimate created successfully!")
    print(f"📁 Output folder: {output_dir}")
    print(f"📄 PDF file: {pdf_name}")
    print(f"🏢 Entity: {entity_sel or 'Any'}")
    print(f"💰 SCS Fee Ranges Applied:")
    print(f"  - Professional Services: $19k-$22k")
    print(f"  - Anesthesia Services: $2k-$4k")
    print(f"  - Facility Services: $74k-$78k")
    
    return result

# Uncomment the line below to use the simple version instead of widgets
# generate_fee_estimate_simple()
```
